***GENERATED CODE FOR customerchurnfe PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs = HDFileSystem(
            host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            df = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***CONNECTOR FUNCTIONS TO WRITE DATA.***

In [ ]:
import datetime
import requests
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class NumtraConnector:

    def put(inStages, inStagesData, stageId, spark, config):
        path = eval(config)['server_url']
        baseType = eval(config)['baseType']
        results_url = eval(config)['results_url']
        server = eval(config)['server']
        originalfile = eval(config)['orignalKey']
        eval(config)['pathOnly']
        filename = eval(config)['filename']
        eval(config)['ser']
        eval(config)['user']
        eval(config)['password']
        eval(config)['authSource']
        eval(config)['user_id']
        eval(config)['parent_id']
        eval(config)['project_id']
        time = str(int(datetime.datetime.now().timestamp()))

        inStagesData[inStages[0]]

        print(path)
        print(baseType)
        print(results_url)
        print(server)
        print(originalfile)
        print(filename)

        args = {
            'url': path,
            'baseType': baseType,
            'originalfile': originalfile,
            'filename': time + filename
        }

        response = requests.post(results_url, args)
        return response


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://0.0.0.0:0000').getOrCreate()
#%run customerchurnfeHooks.ipynb
try:
	#sourcePreExecutionHook()

	customerchurn = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/CustomerChurn.csv', 'filename': 'CustomerChurn.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/', 'results_url': 'http://ml.numtra.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run customerchurnfeHooks.ipynb
try:
	#transformationPreExecutionHook()

	manualfe = TransformationMain.run(customerchurn,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "customerID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "0021-IKXGC", "max": "9967-ATRFS", "missing": "0", "distinct": "500"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "customerID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Female", "max": "Male", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "gender"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SeniorCitizen", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.17", "stddev": "0.37", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "SeniorCitizen"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Partner", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Partner"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Dependents", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Dependents"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "tenure", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "31.45", "stddev": "24.42", "min": "0", "max": "72", "missing": "0"}, "updatedLabel": "tenure"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PhoneService", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PhoneService"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "MultipleLines", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "MultipleLines"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "InternetService", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "DSL", "max": "No", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "InternetService"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "OnlineSecurity", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "OnlineSecurity"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "OnlineBackup", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "OnlineBackup"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DeviceProtection", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "DeviceProtection"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TechSupport", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "TechSupport"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "StreamingTV", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "StreamingTV"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "StreamingMovies", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "StreamingMovies"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Contract", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Month-to-month", "max": "Two year", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Contract"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PaperlessBilling", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PaperlessBilling"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PaymentMethod", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Bank transfer (automatic)", "max": "Mailed check", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PaymentMethod"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "MonthlyCharges", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "65.97", "stddev": "29.55", "min": "18.95", "max": "116.05", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "MonthlyCharges"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TotalCharges", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "499", "mean": "2223.41", "stddev": "2198.76", "min": "19.15", "max": "8468.2", "missing": "1"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "TotalCharges"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Churn", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Churn"}]}))

	#transformationPostExecutionHook(manualfe)

except Exception as ex: 
	logging.error(ex)


***WRITING DATAFRAME***

In [ ]:
#%run customerchurnfeHooks.ipynb
try:
	#sinkPreExecutionHook()

	retailchurn = NumtraConnector.fetch(spark, "{'samplefile': '/FileStore/platform/sampleData/64a69c9e1ba55170f7a2090e/part-00000-b011829f-5dae-4ba3-970a-cd9390aba477-c000.csv', 'samplecount': 759, 'originalcount': 7043, 'orignalKey': '/FileStore/platform/uploadedSourceFiles/CustomerChurn.csv', 'pathOnly': '/Retail Churn', 'project_id': '649c0ff6e61a96675bcdf706', 'parent_id': '649c0ff6e61a96675bcdf706', 'original_schema': [{'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '64a6a18199a38b8079a2ff6d', 'field': 'customerID', 'alias': 'customerID', 'type': 'string', 'position': '0'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '64a6a18199a38b8079a2ff6e', 'field': 'gender', 'alias': 'gender', 'type': 'string', 'position': '1'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '64a6a18199a38b8079a2ff6f', 'field': 'SeniorCitizen', 'alias': 'SeniorCitizen', 'type': 'numeric', 'position': '2'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '64a6a18199a38b8079a2ff70', 'field': 'Partner', 'alias': 'Partner', 'type': 'string', 'position': '3'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '64a6a18199a38b8079a2ff71', 'field': 'Dependents', 'alias': 'Dependents', 'type': 'string', 'position': '4'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '64a6a18199a38b8079a2ff72', 'field': 'tenure', 'alias': 'tenure', 'type': 'numeric', 'position': '5'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '64a6a18199a38b8079a2ff73', 'field': 'PhoneService', 'alias': 'PhoneService', 'type': 'string', 'position': '6'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '64a6a18199a38b8079a2ff74', 'field': 'MultipleLines', 'alias': 'MultipleLines', 'type': 'string', 'position': '7'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '64a6a18199a38b8079a2ff75', 'field': 'InternetService', 'alias': 'InternetService', 'type': 'string', 'position': '8'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '64a6a18199a38b8079a2ff76', 'field': 'OnlineSecurity', 'alias': 'OnlineSecurity', 'type': 'string', 'position': '9'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '64a6a18199a38b8079a2ff77', 'field': 'OnlineBackup', 'alias': 'OnlineBackup', 'type': 'string', 'position': '10'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '64a6a18199a38b8079a2ff78', 'field': 'DeviceProtection', 'alias': 'DeviceProtection', 'type': 'string', 'position': '11'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '64a6a18199a38b8079a2ff79', 'field': 'TechSupport', 'alias': 'TechSupport', 'type': 'string', 'position': '12'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '64a6a18199a38b8079a2ff7a', 'field': 'StreamingTV', 'alias': 'StreamingTV', 'type': 'string', 'position': '13'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '64a6a18199a38b8079a2ff7b', 'field': 'StreamingMovies', 'alias': 'StreamingMovies', 'type': 'string', 'position': '14'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '64a6a18199a38b8079a2ff7c', 'field': 'Contract', 'alias': 'Contract', 'type': 'string', 'position': '15'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '64a6a18199a38b8079a2ff7d', 'field': 'PaperlessBilling', 'alias': 'PaperlessBilling', 'type': 'string', 'position': '16'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '64a6a18199a38b8079a2ff7e', 'field': 'PaymentMethod', 'alias': 'PaymentMethod', 'type': 'string', 'position': '17'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '64a6a18199a38b8079a2ff7f', 'field': 'MonthlyCharges', 'alias': 'MonthlyCharges', 'type': 'real', 'position': '18'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '64a6a18199a38b8079a2ff80', 'field': 'TotalCharges', 'alias': 'TotalCharges', 'type': 'real', 'position': '19'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '64a6a18199a38b8079a2ff81', 'field': 'Churn', 'alias': 'Churn', 'type': 'string', 'position': '20'}], 'actual_schema': [{'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '64a6a18199a38b8079a2ff58', 'field': 'customerID', 'alias': 'customerID', 'type': 'String', 'position': '0'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '64a6a18199a38b8079a2ff59', 'field': 'gender', 'alias': 'gender', 'type': 'String', 'position': '1'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '64a6a18199a38b8079a2ff5a', 'field': 'SeniorCitizen', 'alias': 'SeniorCitizen', 'type': 'numeric', 'position': '2'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '64a6a18199a38b8079a2ff5b', 'field': 'Partner', 'alias': 'Partner', 'type': 'String', 'position': '3'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '64a6a18199a38b8079a2ff5c', 'field': 'Dependents', 'alias': 'Dependents', 'type': 'String', 'position': '4'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '64a6a18199a38b8079a2ff5d', 'field': 'tenure', 'alias': 'tenure', 'type': 'numeric', 'position': '5'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '64a6a18199a38b8079a2ff5e', 'field': 'PhoneService', 'alias': 'PhoneService', 'type': 'String', 'position': '6'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '64a6a18199a38b8079a2ff5f', 'field': 'MultipleLines', 'alias': 'MultipleLines', 'type': 'String', 'position': '7'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '64a6a18199a38b8079a2ff60', 'field': 'InternetService', 'alias': 'InternetService', 'type': 'String', 'position': '8'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '64a6a18199a38b8079a2ff61', 'field': 'OnlineSecurity', 'alias': 'OnlineSecurity', 'type': 'String', 'position': '9'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '64a6a18199a38b8079a2ff62', 'field': 'OnlineBackup', 'alias': 'OnlineBackup', 'type': 'String', 'position': '10'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '64a6a18199a38b8079a2ff63', 'field': 'DeviceProtection', 'alias': 'DeviceProtection', 'type': 'String', 'position': '11'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '64a6a18199a38b8079a2ff64', 'field': 'TechSupport', 'alias': 'TechSupport', 'type': 'String', 'position': '12'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '64a6a18199a38b8079a2ff65', 'field': 'StreamingTV', 'alias': 'StreamingTV', 'type': 'String', 'position': '13'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '64a6a18199a38b8079a2ff66', 'field': 'StreamingMovies', 'alias': 'StreamingMovies', 'type': 'String', 'position': '14'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '64a6a18199a38b8079a2ff67', 'field': 'Contract', 'alias': 'Contract', 'type': 'String', 'position': '15'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '64a6a18199a38b8079a2ff68', 'field': 'PaperlessBilling', 'alias': 'PaperlessBilling', 'type': 'String', 'position': '16'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '64a6a18199a38b8079a2ff69', 'field': 'PaymentMethod', 'alias': 'PaymentMethod', 'type': 'String', 'position': '17'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '64a6a18199a38b8079a2ff6a', 'field': 'MonthlyCharges', 'alias': 'MonthlyCharges', 'type': 'real', 'position': '18'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '64a6a18199a38b8079a2ff6b', 'field': 'TotalCharges', 'alias': 'TotalCharges', 'type': 'real', 'position': '19'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '64a6a18199a38b8079a2ff6c', 'field': 'Churn', 'alias': 'Churn', 'type': 'String', 'position': '20'}], 'server': 'https://ml.numtra.com:443', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/', 'delimiter': ',', 'file_type': 'Delimeted', 'filename': 'FECustomerChurn.csv', 'token': '', 'domain': '', 'is_header': 'Use Header Line', 'url': '/FileStore/platform/uploadedSourceFiles/part-00000-03a41812-9aaf-4422-94ab-5c77d4894274-c000.csv', 'results_url': 'http://ml.numtra.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
